# Install Packages

In [ ]:
#!pip install watson-developer-cloud

In [ ]:
#!pip install PyPDF2

In [ ]:
#!pip install nltk --upgrade

## Import the required libraries.

In [ ]:
import watson_developer_cloud
from watson_developer_cloud import NaturalLanguageUnderstandingV1
from watson_developer_cloud.natural_language_understanding_v1 \
  import Features, EntitiesOptions, KeywordsOptions, SemanticRolesOptions
from watson_developer_cloud import WatsonException 

from io import BytesIO
import zipfile
from collections import OrderedDict
import nltk
from nltk import word_tokenize,sent_tokenize,ne_chunk
import json


from PyPDF2 import PdfFileReader, PdfFileWriter

import nltk

In [ ]:

nltk.download()

In [ ]:
# insert your credentials
natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2018-03-23',
    username="",
    password="")

In [ ]:
# right-click data_doctor.zip, add it as a streaming body and name it as streaming_body_10




In [ ]:
#Global Variables
feature_s=[]
file_name='train.csv' # Name of csv output

### Functions

In [ ]:
def extractZipFilesAndGetContent():
    'extracting data from the zip files'
    zip_ref = zipfile.ZipFile(BytesIO(streaming_body_10.read()),'r')
    paths = zip_ref.namelist()
    req_path=dict()
    temp=[]
    check=''
    content=''
    factsheet=''
    for path in paths:
        factsheet=zip_ref.extract(path)
        print(factsheet)
        test=path.split('/')
        print(test)
        if paths.index(path) < len(paths)-2:
            if len(test) == 4:
                if not test[3]:
                    temp.append(paths[paths.index(path)+1])
                    if 'lab' in paths[paths.index(path)+2]:
                        temp.append(paths[paths.index(path)+2])
                    req_path[test[2]]= temp
                    temp=[]
    zip_ref.close() 
    pdf = PdfFileReader(open(factsheet,'rb'))
    'Extract text from page.'
    for i in range(0, pdf.getNumPages()):
        content += pdf.getPage(i).extractText() + "\n"
    content = " ".join(content.replace("\xa0", " ").strip().split())
    return req_path,content



def analyze_using_NLU(analysistext):
    """ Call Watson Natural Language Understanding service to obtain analysis results.
    """
    response = natural_language_understanding.analyze( 
        text=analysistext,
        features=Features(entities= EntitiesOptions(), keywords=KeywordsOptions(), semantic_roles=SemanticRolesOptions()))
    return response

def extractFeatures():
    entity=[]
    keyword=[]
    semantic=[]
    sample_txt=[]
    subject=[]
    objects=[]
    attribute=[]

    '''Put all the text that has symptoms or symptom keyword into sample_txt'''

    for i in sent_text:
        if "symptoms" in i:
            sample_txt.append(i)
        elif "symptom" in i:
            sample_txt.append(i)

    '''Extract entities,keywords,semantic rule'''

    for x in sample_txt:
        entity= entity+ analyze_using_NLU(x)['entities']
        keyword= keyword+analyze_using_NLU(x)['keywords']
        semantic= semantic+analyze_using_NLU(x)['semantic_roles']

#     '''Find subject,object and action from semantic rules'''    
#     for i in semantic:
#         print("subject:",i['subject']['text'])
#         print("object:",i['object']['text'])
#         print("action:",i['action']['text'])
#         print(i['subject']['text'],i['action']['text'],i['object']['text'])
#         print('-------------------------------------------------------------------------------------------------------------------------')

    '''Find object containing symptoms or symptom as text'''  

    for i in semantic:
        if i['object']['text'] == "symptoms" or i['object']['text'] == "symptom":
            subject.append(i['subject']['text'])
            objects.append(i['object']['text'])

    for k in subject:
        attribute.append(k.split(','))
    '''This is the feature set'''

    for i in attribute:
        for j in i:
            if 'and' in j:
                j=j.replace('and','')
                j=j.lstrip()
                feature_s.append(j)
            else:
                j=j.lstrip()
                feature_s.append(j)
    return feature_s

def lab_reports(path):
    global tests
    '''Feed your pdf file'''
    pdf = PdfFileReader(open(path,'rb'))

    '''Store the content in a variable'''
    content=''
    for i in range(0, pdf.getNumPages()):
        content += pdf.getPage(i).extractText() + "\n"

    '''Algorithm to extract key value pairs'''
    sent_text = nltk.sent_tokenize(content) 
    req=sent_text[3]
    req=req.split('Interpretation')[0]
    req=req.splitlines()
    req= [x.strip() for x in req]
    req=list(filter(None,req))
    matching=[]
    for i in range(len(tests)):
        matching= matching +([s for s in req if tests[i].lower() in s.lower()])
    start_index=req.index(matching[0])
    test=dict()
    f=False
    key=[]
    values=[]
    value=''
    for i in range(start_index,len(req)):
        if req[i] in matching:
            if i!=start_index:
                values.append(value)
            value=''
            key.append(req[i])
            f=True
        elif f and req[i] not in matching:
            if value != '':
                value=value+' '+req[i]
            else:
                value=value+req[i]
    if value != '':
        values.append(value)
    test=dict(zip(key,values))
    tests=[]
    return test


def doctor_reports(path):
    global tests
    '''Feed your pdf file'''
    pdf = PdfFileReader(open(path,'rb'))

    '''Store the content in a variable'''
    content1=''
    val=pdf.getNumPages()
    for i in range(0, pdf.getNumPages()):
        content1 += pdf.getPage(i).extractText() + "\n"

    '''Split the content based on lines'''
    req1=content1.strip().splitlines()
    req1= [x.strip() for x in req1]
    req1=list(filter(None,req1))

    '''Algorithm to extract key value pairs'''
    req_keys=['Patient Name','Age','Details','Symptoms','Tests','Lab report','Lab Id','Recommendation']
    start=0
    end=len(req_keys)
    flag = False 
    final_keys=[]
    temp_values=[]
    final_values=[]
    count = True
    for i in req1:
        if start < end:  
            if req_keys[start] in i:
                final_values.append(temp_values)
                temp_values=[]
                flag = True
                final_keys.append(i)
                start=start+1
            elif flag:
                temp_values.append(i)
        else:
            if req_keys[end-1] in i and f:
                final_keys.append(i)
                f = False
            else:
                temp_values.append(i)
    final_values.append(temp_values)      
    final_values.pop(0)
    docFeatures=dict()
    docFeaturesTemp=dict(zip(final_keys,final_values))
    for k,v in docFeaturesTemp.items():
        if not v:
            items=k.split(':')
            docFeatures[items[0].strip()]=items[1]
        else:
            k=k.replace(':','')
            for z in v:
                if ':' == z:
                    v.remove(z)
            docFeatures[k.strip()]=v



    '''This is the final dictionary'''
    docFeatures['Recommendation']="".join(docFeatures['Recommendation'])
    tests=docFeatures['Tests']
    lab=docFeatures['Lab report'][0]
    symptoms=docFeatures['Symptoms']
    recommendation=docFeatures['Recommendation']
    return lab,symptoms,recommendation


def integration(doc_symptoms,doc_tests,recommendation):
    features_and_values=OrderedDict()
    feature_exists=[] 
    for i in doc_symptoms:
        feature_exists=i.split(',')
    for i in feature_s:
        if i in feature_exists:
            features_and_values[i]="Yes"
        else:
            features_and_values[i]="No"
    if doc_tests != 'null':
        for k,v in doc_tests.items():
            features_and_values[k]=v
    features_and_values['Recommendation']=recommendation
    return features_and_values


def createDataframeFromExtractedData():
    dataframe=[]
    keys=[]
    temp=[]

    for k,v in req_path.items():
        lab,doc_symptoms,recommendation=doctor_reports(v[0])
        doc_tests="null"
        if lab == "Yes":
            doc_tests=lab_reports(v[1])
        features_and_values=integration(doc_symptoms,doc_tests,recommendation)
        dataframe.append(features_and_values)
        

    for d in dataframe:
        if set(d.keys())-set(temp) :
            temp=temp+list(OrderedDict.fromkeys(d))
        
        
    temp=list(OrderedDict.fromkeys(temp))
    df = pd.DataFrame(dataframe, columns=temp)
    return df




### Processing

In [ ]:
req_path, content = extractZipFilesAndGetContent()

In [ ]:
#content

In [ ]:
sent_text = nltk.sent_tokenize(content) 
#sent_text

In [ ]:
feature_s = extractFeatures()

In [ ]:
DataFrame = createDataframeFromExtractedData()
DataFrame

## Save Results to csv

In [ ]:
csv_file=res.to_csv(file_name, sep='\t', encoding='utf-8')
path='/home/dsxuser/work'+file_name

In [ ]:
def put_file(filename, filecontents):
    '''Write file to Cloud Object Storage'''
    resp = cos.put_object(Bucket=credentials_1['BUCKET'], Key=filename, Body=filecontents)
    return resp
put_file(file_name, path)